_한국어로 기계번역됨_



# LangGraph (함수형 API)와 AutoGen, CrewAI 및 기타 프레임워크 통합 방법

LangGraph는 에이전트 기반 및 다중 에이전트 애플리케이션을 구축하기 위한 프레임워크입니다. LangGraph는 다른 에이전트 프레임워크와 쉽게 통합할 수 있습니다.

LangGraph를 다른 에이전트 프레임워크와 통합하고자 하는 주요 이유는 다음과 같습니다:

- 다양한 프레임워크로 구축된 개별 에이전트를 포함하는 [다중 에이전트 시스템](../../concepts/multi_agent)을 생성
- LangGraph를 활용하여 [지속성](../../concepts/persistence), [스트리밍](../../concepts/streaming), [단기 및 장기 기억](../../concepts/memory)과 같은 기능 추가

다른 프레임워크의 에이전트를 통합하는 가장 간단한 방법은 LangGraph의 [노드](../../concepts/low_level/#nodes) 내에서 해당 에이전트를 호출하는 것입니다:

```python
import autogen
from langgraph.func import entrypoint, task

autogen_agent = autogen.AssistantAgent(name="assistant", ...)
user_proxy = autogen.UserProxyAgent(name="user_proxy", ...)

@task
def call_autogen_agent(messages):
    response = user_proxy.initiate_chat(
        autogen_agent,
        message=messages[-1],
        ...
    )
    ...


@entrypoint()
def workflow(messages):
    response = call_autogen_agent(messages).result()
    return response


workflow.invoke(
    [
        {
            "role": "user",
            "content": "피보나치 수열에서 10과 30 사이의 숫자를 찾아라",
        }
    ]
)
```

이 가이드에서는 AutoGen과 통합된 LangGraph 챗봇을 구축하는 방법을 보여주지만, 다른 프레임워크와도 동일한 접근 방식을 따를 수 있습니다.


## 설정


In [1]:
%pip install autogen langgraph


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


OPENAI_API_KEY:  ········


## AutoGen 에이전트 정의

여기에서 우리의 AutoGen 에이전트를 정의합니다. 공식 튜토리얼에서 수정하였습니다 [여기](https://github.com/microsoft/autogen/blob/0.2/notebook/agentchat_web_info.ipynb).


In [3]:
import autogen
import os

config_list = [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

autogen_agent = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.",
)


---


## 워크플로우 생성

이제 AutoGen 에이전트를 호출하는 LangGraph 챗봇 그래프를 생성할 것입니다.


In [8]:
from langchain_core.messages import convert_to_openai_messages, BaseMessage
from langgraph.func import entrypoint, task
from langgraph.graph import add_messages
from langgraph.checkpoint.memory import MemorySaver


@task
def call_autogen_agent(messages: list[BaseMessage]):
    # convert to openai-style messages
    messages = convert_to_openai_messages(messages)
    response = user_proxy.initiate_chat(
        autogen_agent,
        message=messages[-1],
        # pass previous message history as context
        carryover=messages[:-1],
    )
    # get the final response from the agent
    content = response.chat_history[-1]["content"]
    return {"role": "assistant", "content": content}


# add short-term memory for storing conversation history
checkpointer = MemorySaver()


@entrypoint(checkpointer=checkpointer)
def workflow(messages: list[BaseMessage], previous: list[BaseMessage]):
    messages = add_messages(previous or [], messages)
    response = call_autogen_agent(messages).result()
    return entrypoint.final(value=response, save=add_messages(messages, response))


## 그래프 실행하기

이제 그래프를 실행할 수 있습니다.


In [10]:
# pass the thread ID to persist agent outputs for future interactions
# highlight-next-line
config = {"configurable": {"thread_id": "1"}}

for chunk in workflow.stream(
    [
        {
            "role": "user",
            "content": "Find numbers between 10 and 30 in fibonacci sequence",
        }
    ],
    # highlight-next-line
    config,
):
    print(chunk)


user_proxy (to assistant):

Find numbers between 10 and 30 in fibonacci sequence

--------------------------------------------------------------------------------
assistant (to user_proxy):

To find numbers between 10 and 30 in the Fibonacci sequence, we can generate the Fibonacci sequence and check which numbers fall within this range. Here's a plan:

1. Generate Fibonacci numbers starting from 0.
2. Continue generating until the numbers exceed 30.
3. Collect and print the numbers that are between 10 and 30.

Let's implement this in Python:

```python
# filename: fibonacci_range.py

def fibonacci_sequence():
    a, b = 0, 1
    while a <= 30:
        if 10 <= a <= 30:
            print(a)
        a, b = b, a + b

fibonacci_sequence()
```

This script will print the Fibonacci numbers between 10 and 30. Please execute the code to see the result.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python

LangGraph의 [지속성](https://langchain-ai.github.io/langgraph/concepts/persistence/) 기능을 활용하고 있으므로, 이제 동일한 스레드 ID를 사용하여 대화를 계속할 수 있습니다. LangGraph는 이전 기록을 자동으로 AutoGen 에이전트에 전달합니다.


In [12]:
for chunk in workflow.stream(
    [
        {
            "role": "user",
            "content": "Multiply the last number by 3",
        }
    ],
    # highlight-next-line
    config,
):
    print(chunk)


user_proxy (to assistant):

Multiply the last number by 3
Context: 
Find numbers between 10 and 30 in fibonacci sequence
The Fibonacci numbers between 10 and 30 are 13 and 21. 

These numbers are part of the Fibonacci sequence, which is generated by adding the two preceding numbers to get the next number, starting from 0 and 1. 

The sequence goes: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ...

As you can see, 13 and 21 are the only numbers in this sequence that fall between 10 and 30.

TERMINATE

--------------------------------------------------------------------------------
assistant (to user_proxy):

The last number in the Fibonacci sequence between 10 and 30 is 21. Multiplying 21 by 3 gives:

21 * 3 = 63

TERMINATE

--------------------------------------------------------------------------------
{'call_autogen_agent': {'role': 'assistant', 'content': 'The last number in the Fibonacci sequence between 10 and 30 is 21. Multiplying 21 by 3 gives:\n\n21 * 3 = 63\n\nTERMINATE'}}
{'workflow': {'